In [48]:
import requests
import bs4
from bs4 import BeautifulSoup
import csv

from selenium import webdriver
from selenium.common.exceptions import InvalidSessionIdException

def get_url(search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_2'
    
    search_term = search_term.replace(' ', '+') 
    
    # add term query to url
    url = template.format(search_term)
    
    # add page query placeholder
    url += '&page{}'
    
    return url

def extract_record(item):
    """Extract and return data from a single record"""
    
    # description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        # price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        # rating
        rating_parent = item.find('i', {'class': 'a-icon a-icon-star-small a-star-small-4-5 aok-align-bottom'})
        rating = rating_parent.find('span', {'class': 'a-icon-alt'}).text    
    except AttributeError:
        rating = ''   
    
    try:
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:        
        review_count = ''       
    
    result = (description, price, rating, review_count, url)
    
    return result

def main(search_term):
    """Run main program routine"""
    # start up the webdriver 
    ## Chrome
    driver = webdriver.Chrome('C:\\Drivers\\Webdrivers\\chromedriver.exe')
    
    record = []
    records = []
    url = get_url(search_term)
      
    
    for page in (1,21):
           
        driver.get(url.format(page))   
        soup = BeautifulSoup(driver.page_source, 'html.parser')        
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
        driver.close()
        
        #save data to csv file
        with open('results.csv', 'w', newline='', encoding='utf-8') as f:
            write = csv.writer(f)
            write.writerow(['Description', 'Price', 'Rating', 'Review Count', 'URL'])
            write.writerows(records)

In [49]:
main('ultrawide monitor')

InvalidSessionIdException: Message: invalid session id
